<a href="https://colab.research.google.com/github/giuseppefutia/ml/blob/master/gcn_numpy_vs_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Understanding Graph Convolutional Networks using Numpy and Pytorch

In this tutorial I provide some basic concepts on neural networks and I illustrate the graph convolutional network architecture, providing examples using the computational library numpy and the deep learning framework pytorch.

Exploring the numpy code, you are able to fully understand the computations and the algorithm (for instance the backpropagation), while in the Pytorch code you use training algorithms and optimizers as a black box.

I try to provide some autoconsistent examples: in this way you can run each block code autonomusly.

## Two-layered Network
In this first example, we see how to train a shallow neural network with numpy and pytorch.

This example is written starting from the code published at the following link: https://pytorch.org/tutorials/beginner/pytorch_with_examples.html

### Numpy implementation


In [0]:
import numpy as np

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = np.random.randn(N, D_in)
y = np.random.randn(N, D_out)

# Randomly initialize weights
w1 = np.random.randn(D_in, H)
w2 = np.random.randn(H, D_out)

# Define the learning rate 
learning_rate = 1e-6

# Loop for the number of epoch
for t in range(500):
    
    # Forward pass: compute predicted y
    h = x.dot(w1)
    h_relu = np.maximum(h, 0)
    y_pred = h_relu.dot(w2)

    # Compute the Mean Squared Error (MSE) and print the loss.
    loss = np.square(y_pred - y).sum()
    
    if (t % 100) == 0:
      print('epoch:' + str(t), 'loss:' + str(loss))

    # Backprop to compute gradients of w1 and w2 with respect to loss
    # Remember that backprop is performed applying the chain rule of derivatives
    
    # Let's analyze each step of the backprop
    
    # This the derivative of the MSE function
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.T.dot(grad_y_pred)
    grad_h_relu = grad_y_pred.dot(w2.T)
    grad_h = grad_h_relu.copy()
    grad_h[h < 0] = 0
    grad_w1 = x.T.dot(grad_h)

    # Update weights
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

epoch:0 loss:28426790.467448108
epoch:100 loss:573.230085711877
epoch:200 loss:6.783543314221504
epoch:300 loss:0.135135595572394
epoch:400 loss:0.002993381668289992


### Pytorch Implementation

#### Tensors

In this example we adopt pytorch tensors for input, output data, and weights.

Here, we manually perform forward and backward (as we made in the previous step with numpy).

In [1]:
import torch


dtype = torch.float
device = torch.device("cpu")

if torch.cuda.is_available(): # Try to use GPU if available
  device = torch.device('cuda')

print(device)

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random input and output data
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(D_in, H, device=device, dtype=dtype)
w2 = torch.randn(H, D_out, device=device, dtype=dtype)

# Define the learning rate
learning_rate = 1e-6

for t in range(500):
  
    # Forward pass: compute predicted y
    # mm() performs a matrix multiplication of x and w1
    h = x.mm(w1)
    # clam() allows you to construct the relu. The output will be the max betwwen 0 and the number
    h_relu = h.clamp(min=0)
    y_pred = h_relu.mm(w2)

    # Compute and print loss
    # Using item, I can get the effective value of loss (otherwise I will get a tensor)
    loss = (y_pred - y).pow(2).sum().item()
    
    if (t % 100) == 0:
      print('epoch:' + str(t), 'loss:' + str(loss))

    # Backprop to compute gradients of w1 and w2 with respect to loss
    grad_y_pred = 2.0 * (y_pred - y)
    grad_w2 = h_relu.t().mm(grad_y_pred)
    grad_h_relu = grad_y_pred.mm(w2.t())
    grad_h = grad_h_relu.clone()
    grad_h[h < 0] = 0
    grad_w1 = x.t().mm(grad_h)

    # Update weights using gradient descent
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2

cuda
epoch:0 loss:33688504.0
epoch:100 loss:428.52923583984375
epoch:200 loss:1.9298640489578247
epoch:300 loss:0.015833145007491112
epoch:400 loss:0.00035557340015657246


#### Autograd
In this example we do not manually perform forward and backward propagation, but we exploit the autograd feature of Pytorch in order to automate both the steps.

With the autograd function, a computational graph will be built during the forward pass. For each computation a derivative will be generated, that will be used during the backprop.

In [0]:
import torch

dtype = torch.float
device = torch.device("cpu")
if torch.cuda.is_available(): # Try to use GPU if available
  device = torch.device('cuda')

print(device)

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold input and outputs.
# Setting requires_grad=False indicates that we do not need to compute gradients
# with respect to these Tensors during the backward pass.
x = torch.randn(N, D_in, device=device, dtype=dtype)
y = torch.randn(N, D_out, device=device, dtype=dtype)

# Create random Tensors for weights.
# Setting requires_grad=True indicates that we want to compute gradients with
# respect to these Tensors during the backward pass.
w1 = torch.randn(D_in, H, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(H, D_out, device=device, dtype=dtype, requires_grad=True)

# Define the learning rate
learning_rate = 1e-6

for t in range(500):
    # Forward pass: compute predicted y using operations on Tensors; these
    # are exactly the same operations we used to compute the forward pass using
    # Tensors, but we do not need to keep references to intermediate values since
    # we are not implementing the backward pass by hand.
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    # Compute and print loss using operations on Tensors.
    # Now loss is a Tensor of shape (1,)
    # loss.item() gets the a scalar value held in the loss.
    loss = (y_pred - y).pow(2).sum()
    if (t % 100) == 0:
      print('epoch:' + str(t), 'loss:' + str(loss.item()))

    # Use autograd to compute the backward pass. This call will compute the
    # gradient of loss with respect to all Tensors with requires_grad=True.
    # After this call w1.grad and w2.grad will be Tensors holding the gradient
    # of the loss with respect to w1 and w2 respectively.
    loss.backward()

    # Manually update weights using gradient descent. Wrap in torch.no_grad()
    # because weights have requires_grad=True, but we don't need to track this
    # in autograd.
    # An alternative way is to operate on weight.data and weight.grad.data.
    # Recall that tensor.data gives a tensor that shares the storage with
    # tensor, but doesn't track history.
    # You can also use torch.optim.SGD to achieve this.
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        # Manually zero the gradients after updating weights
        w1.grad.zero_()
        w2.grad.zero_()

epoch:0 loss:31386812.0
epoch:100 loss:309.37091064453125
epoch:200 loss:0.6816291809082031
epoch:300 loss:0.0024171480908989906
epoch:400 loss:9.141815098701045e-05


#### nn

Computational graphs and autograd are very powerful to define complex operators and automatically compute derivatives.

Traditionally, when we build neural networks, we usually arrange computation in layers that contain learnable parameters which will be optimized during the learning stage.

In [23]:
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model as a sequence of layers. nn.Sequential
# is a Module which contains other Modules, and applies them in sequence to
# produce its output. Each Linear Module computes output from input using a
# linear function, and holds internal Tensors for its weight and bias.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# The nn package also contains definitions of popular loss functions; in this
# case we will use Mean Squared Error (MSE) as our loss function.
loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-4

for t in range(500):
    # Forward pass: compute predicted y by passing x to the model. Module objects
    # override the __call__ operator so you can call them like functions. When
    # doing so you pass a Tensor of input data to the Module and it produces
    # a Tensor of output data.
    y_pred = model(x)

    # Compute and print loss. We pass Tensors containing the predicted and true
    # values of y, and the loss function returns a Tensor containing the
    # loss.
    loss = loss_fn(y_pred, y)
    if (t % 100) == 0:
      print('epoch:' + str(t), 'loss:' + str(loss.item()))

    # Zero the gradients before running the backward pass.
    # We need to set the gradients to zero before the backward pass, because
    # Pytorch accumulates the gradients on subsequent backward passes. The risk
    # is that gradients can point in some other directions than the minimum.
    model.zero_grad()

    # Backward pass: compute gradient of the loss with respect to all the learnable
    # parameters of the model. Internally, the parameters of each Module are stored
    # in Tensors with requires_grad=True, so this call will compute gradients for
    # all learnable parameters in the model.
    loss.backward()

    # Update the weights using gradient descent. Each parameter is a Tensor, so
    # we can access its gradients like we did before.
    with torch.no_grad():
      for param in model.parameters():
        param -= learning_rate * param.grad

epoch:0 loss:670.6441040039062
epoch:100 loss:2.8842265605926514
epoch:200 loss:0.05636054277420044
epoch:300 loss:0.0021807411685585976
epoch:400 loss:0.00011869626905536279


#### Optim

Until now, the parameters have been manually updated, avoiding tracking history in autograd (using for instance *torch.no_grad()*) .

Nevertheless, if we want to adopt more sophisticated optimizers, we can use the optim package of pytorch as follows.

In [25]:
# -*- coding: utf-8 -*-
import torch

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs
x = torch.randn(N, D_in)
y = torch.randn(N, D_out)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(reduction='sum')

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Tensors it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    if (t % 100) == 0:
      print('epoch:' + str(t), 'loss:' + str(loss.item()))

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable
    # weights of the model). This is because by default, gradients are
    # accumulated in buffers( i.e, not overwritten) whenever .backward()
    # is called. Checkout docs of torch.autograd.backward for more details.
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

epoch:0 loss:627.8787841796875
epoch:100 loss:44.60072326660156
epoch:200 loss:0.4420493543148041
epoch:300 loss:0.0011210866505280137
epoch:400 loss:4.398425971885445e-06


## Graph Convolutional Networks (GCNs)

After an explanation of how neural network architectures are implemented using numpy and pytorh, we are going to explain how to create a neural architecture that is able to learn graph representation for different purposes, from node classification to link prediction.

### What are GCNs
GCNs are a powerful neural network architecture for machine learning. They are able to produce useful feature representations of nodes in networks. 

ADD FURTHER INFORMATION

In [43]:
import numpy as np

# Adjacency Matrix - Size (4,4).
A = np.matrix([
    [0, 1, 0, 0],
    [0, 0, 1, 1], 
    [0, 1, 0, 0],
    [1, 0, 1, 0]],
    dtype=float
)
print('Adjacency Matrix')
print(A)
print()

# Feature Matrix - Size (4.2).
X = np.matrix([
            [i, -i]
            for i in range(A.shape[0])
        ], dtype=float)

print('Feature Matrix')
print(X)
print()

# Applying the propagation rule to obtain an embedding representation of nodes
# from the adjacency matrix. With the following operation, we represent each
# node of the graph with two elements (because of the dimension of the feature matrix).
L1 = A * X
print ('Forward rule')
print(L1)
print()

# Note: The representation of each node (each row) is now a sum of its neighbors
# features. In other words, the graph convolutional layer represents each node 
# as an aggregate of its neighborhood.

# The aggregated representation of a node does not include its own features, for
# these reasons we need to include a self-loop. To compute this self-loop, we
# need to add an identity matrix to the adjacency matrix before applying the
# propagation rules.
I = np.matrix(np.eye(A.shape[0]))
A_hat = A + I
L1 = A_hat * W1
print ('Forward rule with the first loop')
print(L1)
print()

# To avoid vanishing and explosion gradients issues, we need to normalize the
# feature representation of data. We can perform this normalization multiplying
# the adjancecy matrix with the inverse degree matrix. The degree matrix is
# computed summing the values in each adjacency matrix row and then put this
# values in a diagonal matrix

D = np.array(np.sum(A, axis=0))[0]
print('Degree Matrix starting from the Adjacency Matrix')
print(D)
print()
D = np.matrix(np.diag(D))
print('Inverse of Degree Matrix starting from the Adjacency Matrix')
print(D**-1)
print()

# In order to train the network, we need to define a matrix weight
W = np.matrix([
    [1, -1],
    [-1, 1]
])
print('Weight Matrix')
print(W)
print()

# Before comple the propagation we need to create the degree matrix from A_hat
# and not simply A. After that you need to compute the entire propagation
D_hat = np.array(np.sum(A_hat, axis=0))[0]
D_hat = np.matrix(np.diag(D_hat))
forward = D_hat**-1 * A_hat * X * W
print('Forward computation')
print(forward)
print()




Adjacency Matrix
[[0. 1. 0. 0.]
 [0. 0. 1. 1.]
 [0. 1. 0. 0.]
 [1. 0. 1. 0.]]

Feature Matrix
[[ 0.  0.]
 [ 1. -1.]
 [ 2. -2.]
 [ 3. -3.]]

Forward rule
[[ 1. -1.]
 [ 5. -5.]
 [ 1. -1.]
 [ 2. -2.]]

Forward rule with the first loop
[[ 1. -1.]
 [ 6. -6.]
 [ 3. -3.]
 [ 5. -5.]]

Degree Matrix starting from the Adjacency Matrix
[1. 2. 2. 1.]

Inverse of Degree Matrix starting from the Adjacency Matrix
[[1.  0.  0.  0. ]
 [0.  0.5 0.  0. ]
 [0.  0.  0.5 0. ]
 [0.  0.  0.  1. ]]

Weight Matrix
[[ 1 -1]
 [-1  1]]

Forward computation
[[ 1. -1.]
 [ 4. -4.]
 [ 2. -2.]
 [ 5. -5.]]



# Resources


*   [How to do Deep Learning on Graphs with Graph Convolutional Networks](https://towardsdatascience.com/how-to-do-deep-learning-on-graphs-with-graph-convolutional-networks-7d2250723780)
*   List item

